# Imports:

In [ ]:
import os
import time
from urllib.error import HTTPError
import arxiv
from langchain.vectorstores import Qdrant
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOllama
from langchain.embeddings import GPT4AllEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter


en este caso se utiliza la clase arxiv para busqueda de la informacion especifica de su repositorio para guardarlo en local. 

In [ ]:
dirpath = "arxiv_papers"
if not os.path.exists(dirpath):
    os.makedirs(dirpath)
search = arxiv.Search(
    query = "LLM", # your query length is limited by ARXIV_MAX_QUERY_LENGTH which
    max_results = 10,
    sort_by = arxiv.SortCriterion.LastUpdatedDate, # you can also use SubmittedDa
    sort_order = arxiv.SortOrder.Descending
)

Si se encuentra resultados a la busqueda anterior se descarga en formato pdf y lo guarda en la carpeta creada.

In [ ]:
for result in search.results():
    while True:
        try:
            result.download_pdf(dirpath=dirpath)
            print(f"-> Paper id {result.get_short_id()} with title '{result.title}")
            break
        except FileNotFoundError:
            print("File not found")
            break
        except HTTPError:
            print("Forbidden")
            break
        except ConnectionResetError as e:
            print("Connection reset by peer")
            time.sleep(5)

# Split

Carga los datos guardados en la carpeta creada anteriormente y concatena toda su informacion en un mismo string y elimina las lineas vacias.
Luego se utiliza RecursiveCharacterTextSplitter para dividir todo el texto en chunks mas pequeños

In [ ]:
papers = []
loader = DirectoryLoader(dirpath, glob="./*.pdf", loader_cls=PyPDFLoader)
papers = loader.load()
print("Total number of pages loaded:", len(papers)) # Total number of pages loa
full_text = ''
for paper in papers:
    full_text = full_text + paper.page_content

full_text = " ".join(l for l in full_text.splitlines() if l)
print(len(full_text)) # 1466859

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

paper_chunks = text_splitter.create_documents([full_text])

# Add to vectorDB

Se instancia Qdrant como base de datos Vectorial que guardará los chunks con el formato que le da GPT4AllEmbeddings, guardandolo en una carpeta temporal y seleccionando la coleccion a la que pertenece

In [ ]:
qdrant = Qdrant.from_documents(
    documents=paper_chunks,
    embedding=GPT4AllEmbeddings(),
    path="./tmp/local_qdrant",
    collection_name="arxiv_papers",
)

retriever = qdrant.as_retriever()


# Prompt

In [ ]:
#Optionally, pull from the Hub
# from langchain import hub
# prompt = hub.pull("rlm/rag-prompt")
# Or, define your own:

Creamos la plantilla que utilizará el modelo añadiendole el contexto y la pregunta del usuario

In [ ]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM

Seleccionamos un LLm (en este caso se utiliza llama2 pero deve estar descargado en Ollama para poder utilizarlo)

In [ ]:
# Select the LLM that you downloaded
ollama_llm = "llama2:7b-chat"
model = ChatOllama(model=ollama_llm)


# RAG chain

Este código define una canalización de procesamiento utilizando la biblioteca LangChain. Analicemos qué hace cada componente de la tubería:

EjecutableParalelo :

    -Este componente toma dos entradas: "contexto" y "pregunta".
    -Se espera que la entrada de "contexto" la proporcione un recuperador (que no se muestra en el fragmento de código).
    -La entrada de "pregunta" se pasa utilizando el RunnablePassthrough()componente, que simplemente pasa la entrada sin ningún procesamiento.
    -El propósito del RunnableParallelcomponente es ejecutar la recuperación del contexto y el procesamiento de la pregunta en paralelo, mejorando potencialmente el rendimiento general del proceso.

Prompt :

    -Este componente utiliza el ChatPromptTemplateque se definió anteriormente en el código.
    -La plantilla de mensaje se utiliza para formatear el contexto de entrada y la pregunta en un mensaje que pueda ser utilizado por el modelo de lenguaje.

modelo :
    -Este componente representa el modelo de lenguaje que se utilizará para generar la respuesta.
    -En este caso, el modelo de lenguaje es un modelo ChatOllama 

StrOutputParser() :

    -Este componente es responsable de analizar la salida del modelo de lenguaje y convertirla en una cadena.
    
    -El propósito de este componente es garantizar que la salida final de la canalización sea una cadena, que el resto de la aplicación pueda manejar más fácilmente.
    
    -El propósito general de esta canalización es tomar un contexto y una pregunta como entrada, usar un modelo de lenguaje para generar una respuesta basada en el contexto proporcionado y devolver la respuesta como una cadena. El RunnableParallelcomponente se utiliza para ejecutar la recuperación del contexto y el procesamiento de la pregunta en paralelo, lo que potencialmente mejora el rendimiento del proceso.

In [ ]:

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)



# Add typing for input

In [ ]:
class Question(BaseModel):
    __root__: str


In [ ]:
chain = chain.with_types(input_type=Question)
